In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Set up the data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

In [3]:
train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/melanoma10000/Melanoma10000/Original/Train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training')

Found 7684 images belonging to 2 classes.


In [4]:
validation_generator = train_datagen.flow_from_directory(
    '/kaggle/input/melanoma10000/Melanoma10000/Original/Train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation')

Found 1921 images belonging to 2 classes.


In [5]:
# Create the MobileNet-v2 base model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

9406464/9406464 [==============================] - 1s 0us/step


In [6]:
# Create the custom model
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

In [7]:
# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [8]:
# Train the model
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size)

/tmp/ipykernel_24/1662597982.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
240/240 [==============================] - 183s 728ms/step - loss: 0.7138 - accuracy: 0.8170 - val_loss: 0.3087 - val_accuracy: 0.8766
Epoch 2/10
240/240 [==============================] - 128s 536ms/step - loss: 0.3270 - accuracy: 0.8651 - val_loss: 0.2788 - val_accuracy: 0.8854
Epoch 3/10
240/240 [==============================] - 128s 534ms/step - loss: 0.3169 - accuracy: 0.8684 - val_loss: 0.2667 - val_accuracy: 0.8938
Epoch 4/10
240/240 [==============================] - 127s 529ms/step - loss: 0.2994 - accuracy: 0.8758 - val_loss: 0.2397 - val_accuracy: 0.9047
Epoch 5/10
240/240 [==============================] - 126s 526ms/step - loss: 0.2713 - accuracy: 0.8811 - val_loss: 0.2495 - val_accuracy: 0.9010
Epoch 6/10
240/240 [==============================] - 108s 451ms/step - loss: 0.2600 - accuracy: 0.8924 - val_loss: 0.2295 - val_accuracy: 0.9083
Epoch 7/10
240/240 [==============================] - 129s 537ms/step - loss: 0.2477 - accuracy: 0.9008 - val_loss: 0.2540 -

In [9]:
# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/melanoma10000/Melanoma10000/Original/Test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

Found 1000 images belonging to 2 classes.


In [10]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=test_generator.n // test_generator.batch_size)
print('Test accuracy:', test_acc)

/tmp/ipykernel_24/4035526485.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_acc = model.evaluate_generator(test_generator, steps=test_generator.n // test_generator.batch_size)


Test accuracy: 0.9032257795333862
